In [50]:
import doctest
import math
from dataclasses import dataclass
import functools
from typing import Sequence

In [166]:
@dataclass
class PeriodicFractionSqrt:
    a0: int
    a1_to_n: Sequence[int]

    def eval(self, periods: int = 10):
        return (self.a0 +
                functools.reduce(lambda acc, a: 1/(a + acc), 
                                 reversed(self.a1_to_n*periods)))

@dataclass
class PeriodicFractionExpr:
    """
    Represents:     x
               -----------
               sqrt(n) - y
    """
    n: int
    x: int
    y: int

    def next_a(self):
        try:
            return math.floor(self.x * (math.sqrt(self.n) + self.y) / (self.n - self.y**2))
        except:
            return None

    def subtract_a_reciprocal(self, new_a):
        new_x = (self.n - self.y**2) // self.x
        new_y = new_a * new_x - self.y
        return PeriodicFractionExpr(self.n, new_x, new_y)

In [106]:
print(PeriodicFractionSqrt(4, (1, 3, 1, 8)).eval())
print(math.sqrt(23))

4.795831523312719
4.795831523312719


In [107]:
math.floor(1*(math.sqrt(23) + 4) / (23 - 4**2))

1

In [127]:
frac = PeriodicFractionExpr(23, 1, 4)
for i in range(8):
    next_a = frac.next_a()
    print(f'a_{i}:', next_a)
    frac = frac.subtract_a_reciprocal(next_a)

a_0: 1
a_1: 3
a_2: 1
a_3: 8
a_4: 1
a_5: 3
a_6: 1
a_7: 8


In [167]:
def has_cycle(xs):
    """
    >>> has_cycle([])
    False
    >>> has_cycle([1, 4, 1, 2])
    False
    >>> has_cycle([1, 4, 4, 1])
    False
    >>> has_cycle([5, 5])
    True
    >>> has_cycle([1, 4, 1, 2, 1, 4, 1, 2])
    True
    """
    n = len(xs)
    return n > 0 and n % 2 == 0 and xs[:n//2] == xs[n//2:]

def sqrt_periodic_form(n):
    a0 = math.floor(math.sqrt(n))
    expr0 = PeriodicFractionExpr(n, 1, a0)
    a1_to_n = []

    prev_a = a0
    prev_expr = expr0
    # Continue until we reach a cycle.
    while prev_expr != expr0 or len(a1_to_n) == 0:
        next_a = prev_expr.next_a()
        if next_a is None: break
        next_expr = prev_expr.subtract_a_reciprocal(next_a)
        a1_to_n.append(next_a)
        prev_a, prev_expr = next_a, next_expr

    return PeriodicFractionSqrt(a0, a1_to_n)

In [163]:
for n in [2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 23]:
    print(n, ':', sqrt_periodic_form(n))

2 : PeriodicFractionSqrt(a0=1, a1_to_n=[2])
3 : PeriodicFractionSqrt(a0=1, a1_to_n=[1, 2])
5 : PeriodicFractionSqrt(a0=2, a1_to_n=[4])
6 : PeriodicFractionSqrt(a0=2, a1_to_n=[2, 4])
7 : PeriodicFractionSqrt(a0=2, a1_to_n=[1, 1, 1, 4])
8 : PeriodicFractionSqrt(a0=2, a1_to_n=[1, 4])
10 : PeriodicFractionSqrt(a0=3, a1_to_n=[6])
11 : PeriodicFractionSqrt(a0=3, a1_to_n=[3, 6])
12 : PeriodicFractionSqrt(a0=3, a1_to_n=[2, 6])
13 : PeriodicFractionSqrt(a0=3, a1_to_n=[1, 1, 1, 1, 6])
23 : PeriodicFractionSqrt(a0=4, a1_to_n=[1, 3, 1, 8])


In [172]:
sqrt_periodic_form(16)

PeriodicFractionSqrt(a0=4, a1_to_n=[])

In [170]:
sum(len(sqrt_periodic_form(n).a1_to_n) % 2 == 1 for n in range(2, 10001))

1322

In [175]:
# Out of curiosity...
max((len(sqrt_periodic_form(n).a1_to_n), n) for n in range(2, 10001))

(217, 9949)

In [176]:
sqrt_periodic_form(9949)

PeriodicFractionSqrt(a0=99, a1_to_n=[1, 2, 1, 10, 1, 65, 1, 1, 2, 1, 1, 3, 3, 21, 1, 6, 5, 1, 9, 7, 3, 2, 17, 1, 2, 2, 1, 1, 1, 3, 4, 1, 5, 4, 3, 1, 4, 1, 14, 1, 1, 12, 1, 3, 1, 1, 1, 1, 4, 1, 1, 39, 2, 1, 6, 2, 5, 13, 8, 1, 1, 2, 16, 4, 2, 1, 2, 5, 49, 1, 2, 5, 2, 1, 2, 1, 15, 1, 8, 1, 1, 3, 1, 2, 1, 1, 4, 1, 27, 1, 2, 9, 1, 1, 1, 3, 9, 4, 2, 2, 1, 7, 3, 1, 2, 2, 2, 4, 4, 2, 2, 2, 1, 3, 7, 1, 2, 2, 4, 9, 3, 1, 1, 1, 9, 2, 1, 27, 1, 4, 1, 1, 2, 1, 3, 1, 1, 8, 1, 15, 1, 2, 1, 2, 5, 2, 1, 49, 5, 2, 1, 2, 4, 16, 2, 1, 1, 8, 13, 5, 2, 6, 1, 2, 39, 1, 1, 4, 1, 1, 1, 1, 3, 1, 12, 1, 1, 14, 1, 4, 1, 3, 4, 5, 1, 4, 3, 1, 1, 1, 2, 2, 1, 17, 2, 3, 7, 9, 1, 5, 6, 1, 21, 3, 3, 1, 1, 2, 1, 1, 65, 1, 10, 1, 2, 1, 198])

In [78]:
doctest.testmod(verbose=True)

Trying:
    has_cycle([1, 4, 1, 2])
Expecting:
    False
ok
Trying:
    has_cycle([1, 4, 4, 1])
Expecting:
    False
ok
Trying:
    has_cycle([5, 5])
Expecting:
    True
ok
Trying:
    has_cycle([1, 4, 1, 2, 1, 4, 1, 2])
Expecting:
    True
ok
7 items had no tests:
    __main__
    __main__.PeriodicFractionSqrt
    __main__.PeriodicFractionSqrt.__eq__
    __main__.PeriodicFractionSqrt.__init__
    __main__.PeriodicFractionSqrt.__repr__
    __main__.PeriodicFractionSqrt.eval
    __main__.sqrt_periodic_form
1 items passed all tests:
   4 tests in __main__.has_cycle
4 tests in 8 items.
4 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=4)